# 5章 モノリスの分割

## 5.1 すべては接合部次第

* サービスは凝集性を高め、疎結合にしたい
* モノリスは大抵どちらにも反している
* モノリスはシステム全体をデプロイしなおさなければいけない

* コードベースを整理するために接合部を探すのではなく、サービス境界となる接合部を特定したい

## 5.2 MusicCorpの分解

* MusicCorp: オンラインシステム、大規模なバックエンドモノリシックサービス


* まずは、組織のコンテキスト境界を特定し、モノリスの対応するコンテキスト境界を把握
    * 既存コードをコンテキストを表すパッケージに分類していく
    * IDEでリファクタリングすることが難しい言語を利用している場合は、破壊を捕捉するテストも必要


* 分類できなかったコードから見逃したコンテキストを特定できる


* パッケージ間の依存関係と、実際の組織間の依存関係（対話関係）が合っているかどうか
    * 実際にはやり取りしていない組織なのに、パッケージ間で依存があればおかしい

## 5.3 モノリスを分割する理由

* モノリスの分割は少しずつ進めていくことでマイクロサービスについて学ぶ手助けとなるし、間違いの影響も制限される


* コードベースのどの部分を分離したら最も恩恵が得られるかを考えるのが最善
    * 変化の速度
        * 在庫の管理方法には短時間で大量の変更が発生してしまうことがわかるでしょう（？）
        * よく変更される箇所は独立させたほうがいいということ
    * チーム構成
        * 物理的に離れたところにいるチーム単位で分割するといい → 詳しくは10章
    * セキュリティ
        * 現在はセキュリティ監査のため経理関連のコードですべて対処している
        * このサービスのコードを分割したら、監視、転送データの保護、格納データの保護に関してこの別個のサービスに保護を追加できます（？） → 詳しくは9章
    * 技術
        * 使用する技術要素（プログラミング言語など）の単位でサービス分割
        * そうすると、テスト対象の代替実装の構築を検討しやすくなるでしょう（？）

## 5.4 入り組んだ依存関係

* できれば依存関係の少ない接合部を取り出したい
* 入り組んだ依存関係の原因となることが多いのはデータベース

## 5.5 データベース

* 複数のサービスの統合手法としてデータベースを利用することは勧められない（前の章で説明している）

## 5.6 問題の対処

* データベースアクセスを行う部分を同じようにコンテキスト境界で分割したい
    * Hibernateなどを利用しているリポジトリレイヤを一枚岩ではなく分割したい


* コード上は別のコンテキストのテーブルを使っていることがわかっていても、実際にそのテーブル間でデータベース上の依存関係があるかどうかはわからない
    * SchemaSpy: テーブル間の関係をグラフィカルな表現で確認できる


* テーブル間の結びつきを断ち切るのは簡単ではないが対処は可能
* 次の節から方法を見ていく

## 5.7 例：外部キー関係の削除

* 例: 経理の売上情報で品物の名前を品目テーブルから引きたい
* 修正: 経理パッケージはカタログパッケージのAPI経由で品物の名前を取得する


* 異なるデータベースのアクセスとなるとパフォーマンスの懸念が出てくる
    * 実際に問題となるかどうかをテストすること
    * 必要なパフォーマンスは? 現状のパフォーマンスは?
    * 複数のデータベースアクセスを行っていても、必要なパフォーマンスが確保できているのであれば問題ない


* 外部キー関係が分断される
    * 一貫性検査を実装するか、関連データを整理する措置
    * 多くの場合、技術者が判断することではない
    * 不完全なデータが存在したときにどうするかを、システムの振る舞いを決める人に答えてもらう
    * その上でどんな機能が必要かが明確になる

## 5.8 例：共有静的データ

* 例: 国コード（サポートする国が頻繁に更新される予定）


* 選択肢1: パッケージ/サービスごとにテーブルを重複して持つ
    * 一貫性の課題（一部の更新忘れ）が起こる可能性
* 選択肢2: 共有静的データをコードとして扱う
    * プロパティファイルか列挙型か
    * 稼働中のデータベースを変更するよりもはるかに簡単（？） → デプロイし直す手間のほうが大きいような？

## 5.9 例：共有データ

* 例: 経理コードと倉庫コードがそれぞれ支払いと在庫の管理を同じ顧客レコードテーブルに書き込んでいる
    * 顧客パッケージ/サービスとして切り出せる
    * 外部キー関係の問題が発生する？

## 5.10 例：共有テーブル

* 例: カタログコードが品目の名前と価格、倉庫コードが在庫を管理しており、どちらも品目テーブルに書き込んでいる
    * カタログ品目と在庫水準（？）に分割する
    * やはり外部キー関係の問題が発生する？

## 5.11 データベースリファクタリング

* スキーマの分割 → サービスの分割の順番に行うのがオススメ
    * 2つのスキーマ（データベース）に分割されるとトランザクション完全性（整合性）が失われてしまう
    * いつでも元の構成に戻せる状態で、重大な影響がないかを確認したのちにサービス（アプリケーションコード）を分割する
    * 詳しくは次の節で説明

## 5.12 トランザクション境界

* スキーマを分割するとトランザクションも分割されてしまう


* 5.12.1 後でリトライ
    * 失敗したトランザクションを後からリトライすることでいずれは求めていた状態になるようにする
    * 結果整合性: トランザクション境界を利用してトランザクション完了時にシステムが一貫性のある状態になることを保証する代わりに、システムが将来のある時点で一貫性のある状態になることを受け入れる
    * 詳しくは11章スケーリングのパターンで


* 5.12.2 操作全体の中止
    * 先行して成功してしまったトランザクションを打ち消すような補正（補償）トランザクションを発行
    * 補正トランザクションも失敗するとどうするか → リトライあるいは、バックエンドプロセスで非一貫性を取り除く仕組み
    * 一貫性を保ちたい操作が多くなればなるほど対応は困難


* 5.12.3 分散トランザクション
    * トランザクションマネージャ: 全体のトランザクションを管理するプロセス
    * 2フェーズ（2相）コミット
        * 投票フェーズ: 分散トランザクション参加者がローカルトランザクションを進められるかどうか確認 → 1つでも「いいえ」があれば、全参加者にロールバックを送信
        * 投票の応答に失敗したときに全体が停止してしまう
        * 「はい」と答えたらコミットできるとみなすが、実際は失敗してしまった場合
        * 絶対確実なアルゴリズムではない


* 5.12.4 何をすべきか
    * 分散トランザクション
        * 正しく実行するのが困難
        * スケーリングを妨げる
        * データの非一貫性を修正するために補正動作が必要になる場合がある
    * 結果整合性にできないか
        * この方が構築やスケーリングが簡単 → 詳しくは11章
    * 本当に一貫性を維持したい場合
        * トランザクション自体を表す具体的な概念を作成（例: 処理中の注文）

## 5.13 レポート

* データの格納方法や格納場所を分割するとレポートの作成で問題になる

## 5.14 レポートデータベース

* モノリシックサービスの場合
    * プライマリデータベースから定期的に同期したレポートレプリカを、レポートシステムが利用する


* データベースのスキーマがモノリシックサービスとレポートシステムで共有APIとなる
    * スキーマの変更を慎重にしなければいけない
* サービスとレポートシステムの両方に適した最適化方法が限られる
    * それぞれに最適なデータベースがある
    * 両方ともに最適なものはまずない

## 5.15 サービス呼び出しを介したデータ取得

* 少ないデータだけを必要とするレポートでは、サービス呼び出しを介してデータを取得すればよい
* 大量データを必要とする場合には機能しなくなる
* 公開されているAPIはおそらくレポート向けに設計されていない
    * 顧客をIDで引くことはできても、すべての顧客を取得するAPIはないかもしれない


* バッチ用APIを作成する
* リクエストを受けるとHTTP 202(Accepted)をすぐに返して、共有場所にファイルとして出力する
* 呼び出し側はHTTP 201(Created)が返るまでポーリングしてから、共有場所にファイルを取りに行く

## 5.16 データポンプ

* HTTP呼び出しの欠点
    * 多数の呼び出しを行ったときのオーバーヘッド
    * レポート専用のAPIを作成しなければいけない


* データベースからレポート用のデータを吸い出すスタンドアロンプログラム（データポンプ）
    * レポート専用のデータベースに書き出す
    * 同じデータベースに多くのプログラムがアクセスするべきではないが、これは特筆すべき例外


* データポンプはサービスを管理するのと同じチームが構築・管理すべき
    * サービス向けの内部データベースとレポートスキーマ両方の知識が必要


* レポートスキーマ自体との結合は残る
    * 変更が難しい公開APIとして扱わなければいけない
    * このコストを軽減するテクニック
        * レポートデータベースでは各サービスごとのスキーマを用意
        * レポートツールから使用するスキーマをマテリアライズドビューで実現
        * JBoss Data Virtualizationもこの領域


* 5.16.1 代替手段
    * S3を使った巨大データマート: ソリューションのスケーリングが必要になるまではとてもうまく機能 → なぜスケーリングできない?

## 5.17 イベントデータポンプ

* データが変更されたらイベントが発行され、それに対するイベントサブスクライバがレポートデータベースにデータを書き込む
    * イベントサブスクライバ = レポートマッパー ?


* どのイベントが既に処理済みか格納しておくと差分だけ処理できる
    * レポートデータベースに処理済みイベントを格納?


* イベントデータポンプはサービス内部とあまり結合しないのでサービス担当と別のグループで管理しやすい


* イベントデータベースに必要なデータがすべてイベントに含まれている必要があるという欠点がある
    * 大量のデータが必要な場合はスケーリングがうまくできない場合がある

## 5.18 バックアップデータポンプ

* Netflixはバッキングストア（バックアップ先?）としてCassandraを使用
    * SSTable（Sorted String Table: ソート済み文字列テーブル, Cassandraのファイル形式）をAmazon S3に格納
    * Hadoopを利用してレポートを作成

## 5.19 リアルタイムを目指す

* すべてのレポートを1箇所から行う考え方は本当に妥当なのか
* 詳しくは8章

## 5.20 変更のコスト

* 小さな変更を漸進的に行うこと
* 間違いをなくすことはできないので受け入れること


* 影響の大きいところは分割にコストがかかるし、元に戻すのも大変
* 影響が最も少ないところで間違いを犯すようにする


* CRC（Class-Responsibility-Collaboration、クラス-責務-協調）カード

## 5.21 根本原因の理解

* なぜサービスが大きくなってしまうのか → それ自体に問題はない
* 重要なのは、分割のコストが高くなりすぎる前に分割する必要があるかを判断すること


* 新しいサービスを立ち上げやすくすること
    * セルフサービスで仮想マシンをプロビジョニング、PaaSの利用など
    * 次章から説明していく

## 5.22 まとめ

* 接合部を探してシステムを分割し、漸進的な方法で実行
* 最初の段階でのサービス分割のコストを減らすようにすると、継続的にシステムを成長、進化させられる